## Approach 1. Usar um modelo pre-treinado para encontrar por similaridade no banco

In [11]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity

# Função para carregar e pré-processar imagens
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Arquivo não encontrado: {image_path}")
    print("Carregando imagem:", image_path)  # Adicionando instrução de impressão
    
    # Carregar a imagem com Pillow
    with Image.open(image_path) as img:
        image = np.array(img)
    
    if image is None:
        raise ValueError(f"Erro ao ler a imagem: {image_path}")
    
    # Converter para RGB se necessário
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    else:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Redimensionar a imagem
    image = cv2.resize(image, target_size)
    
    # Pré-processamento final
    image = img_to_array(image)
    image = preprocess_input(image)
    
    return image



# Função para extrair características de uma imagem usando MobileNetV2
def extract_features(image_path, model):
    image = load_and_preprocess_image(image_path)
    features = model.predict(np.expand_dims(image, axis=0))
    return features.flatten()

# Carregar o modelo MobileNetV2 pré-treinado
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Carregar imagens de pets e extrair características
pet_features = []
pet_files = []
for file in os.listdir('imagens'):
    if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png'):
        file_path = os.path.join('imagens', file)
        pet_files.append(file)
        features = extract_features(file_path, base_model)
        pet_features.append(features)

# Convertendo a lista de características em matriz
pet_features = np.array(pet_features)

# Função para encontrar o pet mais semelhante
def find_most_similar_pet(image_path, pet_features, pet_files, base_model):
    query_features = extract_features(image_path, base_model)
    similarities = cosine_similarity([query_features], pet_features)
    most_similar_index = np.argmax(similarities)
    return pet_files[most_similar_index]

Carregando imagem: imagens\AABB São Leopoldo_Rua Cristopher Levalley, 774 - São Borja, São Leopoldo - RS, 93037-730_id_1101.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 976ms/step
Carregando imagem: imagens\Animacão_R. Esmeralda, 500 - Imigrante Sul, Campo Bom - RS, 93700-000_id_1110.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Carregando imagem: imagens\Animacão_R. Esmeralda, 500 - Imigrante Sul, Campo Bom - RS, 93700-000_id_1164.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Carregando imagem: imagens\Animacão_R. Esmeralda, 500 - Imigrante Sul, Campo Bom - RS, 93700-000_id_1182.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Carregando imagem: imagens\Animacão_R. Esmeralda, 500 - Imigrante Sul, Campo Bom - RS, 93700-000_id_1190.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Carregando imagem: imagens\Animacão_R. Esmeralda, 500 - Imigrante Sul, Campo Bom - RS, 93700-000_id_1191.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Carregando imagem: imagens\Animacão_R. Esmeralda, 500 - Imigrante Sul, Campo Bom - RS, 93700-000_i

In [21]:
# Testar com uma imagem de exemplo
query_image_path = r'C:\pethFinder\Notebooks\imagens\Ginásio Municipal Agostinho Emilio Cavasotto_Rincão, Novo Hamburgo - RS, 93348-510_id_2075.jpg'
most_similar_pet = find_most_similar_pet(query_image_path, pet_features, pet_files, base_model)
print("O pet mais semelhante é:", most_similar_pet)

Carregando imagem: C:\pethFinder\Notebooks\imagens\Ginásio Municipal Agostinho Emilio Cavasotto_Rincão, Novo Hamburgo - RS, 93348-510_id_2075.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
O pet mais semelhante é: Ginásio Municipal Agostinho Emilio Cavasotto_Rincão, Novo Hamburgo - RS, 93348-510_id_2075.jpg


Apesar de encontrar a mesma imagem que esta no banco, não possui robustez para outras fotos novas

## Approach 2. Treinar do zero